# Consumer Expenditure Trends 2006-2020 
## Ben Warzel
### 5/3/2025

*Note: generative AI was used in this project during data analysis*

### Introduction
I chose this particular set of consumer expenditure data because it covers the majority of my childhood and adolescence, and I was curious to see the impacts of events like the 2008 housing crisis on the average American's spending habits. The rise and prevalence of credit cards into the 2010s is something else that might affect how people spend. 

According to the Bureau of Labor Statistics, by 2005 consumers were spending 97% of their income. During the Great Recession, that number dropped to 92% due to higher mortage debt and lower home equity. People had to try to save more of their money. The BLS also has data on employment statistics for the same time period, which we will examine later.

The main categories I examined here are Housing, Healthcare, Food, and Transportation. I also attempt to answer questions about extraneous spending; how the consumption of alcohol and tobacco changes over time for example, or the section labeled "Entertainment" which we can assume to mean things like going to the movies or concerts. 